## Introduction to Human In The Loop in Agent Framework

Workflows in the Microsoft Agent Framework allow developers to design, monitor, and control how multiple AI agents and logic components interact to complete complex tasks. They bring structure, flexibility, and transparency to agent-driven applications.

One of the ways to bring in control and manage agent workflows is to add human approvals. When agents require any user input, for example to approve a function call, this is referred to as a **human-in-the-loop** pattern. 

An agent run that requires user input will complete with a response that indicates what input is required from the user instead of completing with a final answer. The caller of the agent is then responsible for getting the required input from the user and passing it back to the agent as part of a new agent run.


### Example 1 - Function Tools with Human-In-The-Loop approvals



In [1]:
import os
from dotenv import load_dotenv
from agent_framework.azure import AzureOpenAIChatClient

load_dotenv()
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

chat_client=AzureOpenAIChatClient(
        endpoint=endpoint,
        api_key=api_key,
        api_version=api_version,
        deployment_name=deployment,
    )

#### Key components

1. `@ai_function(approval_mode="always_require")`:

Use the `@ai_function` decorator to specify metadata. To create a function that requires approval, you can use the `approval_mode` parameter so the agent can pause and ask for user approval.

2. `user_input_requests`:

Since you now have a function that requires approval, the agent can respond with a request for approval instead of executing the function directly and returning the result. 

You should check the response for any user input requests, which indicates that the agent requires user approval for a function. 

The details of the function call including name and arguments can be found in the `function_call` property on the user input request:

```python
    user_input_needed.function_call.name
    user_input_needed.function_call.arguments
```

3. `create_response()`:

Once the user has provided their input, you can create a response using the `create_response` method on the user input request. Pass True to approve the function call, or False to reject it.

The response should then be passed to the agent in a new ChatMessage:

```python
    approval_message = ChatMessage(
    role=Role.USER, 
        contents=[user_input_needed.create_response(user_approval)]
    )
```
`.create_response(user_approval)` embeds the approval decision inside a message the framework understands.

4. **Resuming Execution:**

After collecting user approval decisions for pending function calls, you need to resume the agent's execution by reconstructing the conversation history. To resume execution, you must provide both the agent's function call requests and the user's approval responses as ChatMessage objects:
    
```python
    final_result = await agent1.run([
    "What is the detailed weather like in Amsterdam?",
    ChatMessage(role=Role.ASSISTANT, contents=[user_input_needed]),
    approval_message
    ])
```
By providing both messages, you're reconstructing the conversation flow: the agent proposed an action (ASSISTANT), and the user responded to that proposal (USER).

> *Note*: In this example, if the user rejects a function call (`user_approval = False`), the agent doesn't simply fail. Instead, it attempts to fulfill the user's intent using alternative available tool - `get_weather`. In this example, if `get_weather_detail` is rejected, the agent may fall back to calling `get_weather` (which doesn't require approval) to still provide some weather information.

In [2]:
from typing import Annotated
from agent_framework import ai_function, ChatMessage, Role, ChatAgent 


@ai_function
def get_weather(location: Annotated[str, "The city and state, e.g. San Francisco, CA"]) -> str:
    """Get the current weather for a given location."""
    return f"The weather in {location} is sunny with a high of 30°C."

@ai_function(approval_mode="always_require")
def get_weather_detail(location: Annotated[str, "The city and state, e.g. San Francisco, CA"]) -> str:
    """Get detailed weather information for a given location."""
    return f"The weather in {location} is cloudy with a high of 15°C, humidity 88%."


agent = ChatAgent(
    chat_client=chat_client,
    name="WeatherAgent",
    instructions="You are a helpful weather assistant.",
    tools=[get_weather_detail, get_weather],
)

result = await agent.run("What is the detailed weather like in Amsterdam?")

if result.user_input_requests:
    print("Approval required:")
    for user_input_needed in result.user_input_requests:
        print(f"Function: {user_input_needed.function_call.name}")
        print(f"Arguments: {user_input_needed.function_call.arguments}")
        
    # Get user approval (non-interactive here)
    user_approval = True 

    # Create the approval response
    approval_message = ChatMessage(
        role=Role.USER, 
        contents=[user_input_needed.create_response(user_approval)]
    )

    final_result = await agent.run([
        "What is the detailed weather like in Amsterdam?",
        ChatMessage(role=Role.ASSISTANT, contents=[user_input_needed]),
        approval_message
    ])

print(final_result.text)

ServiceResponseException: <class 'agent_framework.azure._chat_client.AzureOpenAIChatClient'> service failed to complete the prompt: Error code: 403 - {'error': {'code': 'AuthenticationTypeDisabled', 'message': 'Key based authentication is disabled for this resource.'}}

In the example above, during the first agent run, the agent doesn't complete execution because it identifies that `get_weather_detail` which requires approval needs to be called. Instead of executing the function, the first `agent.run()` call completes immediately and returns the pending function call in `result.user_input_requests`.

We then collect the user's approval decision and resume execution via the second `agent.run()` call with the complete conversation history, including the approval response. This processes the approval decision and either executes the function (if approved) or adapts its strategy (if rejected) to provide a final response. This pattern ensures human oversight while giving your application full control over how approval requests are presented to users.

### Example 2
First, let's define our agent functions. 

In [3]:
from agent_framework import ai_function
from typing import Annotated
import random

@ai_function(approval_mode="always_require")
def submit_payment(
    amount: Annotated[float, "Payment amount in USD"],
    recipient: Annotated[str, "Recipient name or vendor ID"],
    reference: Annotated[str, "Short description for the payment reference"],
) -> str:
    """
    Submit a payment request to the external payments system.

    This operation has financial impact and should always be reviewed
    and approved by a human before it is executed.
    """
    # In a real scenario this would call an external payments API.
    # Here we just simulate the side effect.
    return (
        f"Payment of ${amount:.2f} to '{recipient}' has been submitted "
        f"with reference '{reference}'."
    )

@ai_function(
    name="get_account_balance",
    description="Retrieves the current account balance for the user in USD"
)
def get_account_balance() -> float:
    """
    Get the current account balance for the user.
    
    Returns:
        float: The account balance in USD (numeric value only, no formatting).
    
    This operation is read-only and does not require approval.
    """
    # Generate a random balance between 1000 and 5000 USD
    balance = random.uniform(1000, 5000)
    return round(balance, 2)

In [4]:
import os
import asyncio
from agent_framework import ChatAgent, ChatMessage, Role

# Stateful agent wired to Azure OpenAI plus both banking tools
agent = ChatAgent(
    chat_client=chat_client,
    name="FinanceAgent",
    instructions=(
        "You are an agent from Contoso Bank. You assist users with financial operations "
        "and provide clear explanations. For transfers only amount, recipient name, and reference are needed."
    ),
    tools=[submit_payment, get_account_balance],
)

# Preserve conversation memory across the entire console session
thread = agent.get_new_thread()
print("=== FinanceAgent - Interactive Session ===")
print("Type 'exit' or 'quit' to end the conversation\n")

while True:
    user_input = input("You: ").strip()
    if user_input.lower() in ("exit", "quit"):
        print("Goodbye!")
        break
    if not user_input:
        continue

    print("\nAgent: ", end="", flush=True)
    result = await agent.run(user_input, thread=thread)

    if result.user_input_requests:
        print("\n\n=== APPROVALS REQUIRED ===")
        approval_messages: list[ChatMessage] = []

        # Surface every pending tool call and gather a decision per call
        for req in result.user_input_requests:
            print(f"- Function: {req.function_call.name}")
            print(f"  Arguments: {req.function_call.arguments}")
            approved = input(f"Approve '{req.function_call.name}'? (yes/no): ").strip().lower() == "yes"

            # Encode the approval/denial into a ChatMessage the framework consumes
            approval_messages.append(
                ChatMessage(role=Role.USER, contents=[req.create_response(approved)])
            )

        # Resume the prior run once all approvals are ready
        followup = await agent.run(approval_messages, thread=thread, prior_run=result)
        print("\nAgent:", followup.text)
        print("\nUser:", result.text)

    else:
        print(result.text)
    
    print()

print(type(followup))

print("RESULT:",result.messages)
for msg in result.messages:
    print(f" - {msg.role}: {msg.text}")

messages = await thread.message_store.list_messages()
print("FOLLOWUP:", followup)
for msg in messages:
    print(f" - {msg.role}: {msg.text}")

=== FinanceAgent - Interactive Session ===
Type 'exit' or 'quit' to end the conversation


Agent: 

ServiceResponseException: <class 'agent_framework.azure._chat_client.AzureOpenAIChatClient'> service failed to complete the prompt: Error code: 403 - {'error': {'code': 'AuthenticationTypeDisabled', 'message': 'Key based authentication is disabled for this resource.'}}

### Request and Response

#### Recap - Message-Centric Execution Model

In AF, agent execution is fundamentally about the **data flow** - a workflow will keep running as long as there is data being passed. Agents don't "call" each other or pass control in a traditional procedural sense. Instead, they communicate through typed messages flowing between executors through edges.

**Executors** are where computation happens. When an executor emits a message, it flows along the edges to downstream executors which activate when their dependent data becomes available.

Agents communicate with applications via different message content types (`TextContent`, `FunctionCallContent`, `FunctionApprovalRequestContent`, etc.). Messages are represented by the `ChatMessage` class and all content type classes inherit from the base `BaseContent` class.

For a detailed list of available content types and their usage, refer to the folowing [documentation](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/running-agents?pivots=programming-language-python).


#### User Approval Content

The content of the message can also be used to let the application approve or disapprove some actions the agent wants to take.
**User Approval Content** type enables human-in-the-loop approvals within the standard message flow.


The approval mechanism intercepts the normal tool invocation flow, pausing execution until human approval is granted or denied:

<img src="images/reqresp.png" alt="User Approval Content Type" style="width:70%; height:40%; margin:20px auto">

When a ChatAgent needs to invoke a Tool that requires approval:

1. **Initial Inference**: The ChatAgent uses the Model to make an inference and determines it needs to invoke the Tool.

2. **Approval Request**: Before executing, the Tool sends a request back to the ChatAgent, which surfaces to the Application as `FunctionApprovalRequestContent` (a special ChatMessage content type).

3. **User Response**: The Application presents this approval request to the user, who responds (Yes/No). This response is sent back to the ChatAgent as `FunctionApprovalResponseContent`.

4. **Tool Execution**: Once the ChatAgent receives approval, the Tool executes and returns its result to the ChatAgent.

5. **Final Inference**: The Model makes another inference using the Tool's result to generate a natural language response as `TextContent`.

6. **Complete Response**: The Application receives both the `FunctionResultContent` (the raw tool output) and the `TextContent` (the Model's interpretation of that result).

#### User Approval Importance

User approvals are necessary in many cases, especially when tool execution involves external resources or has side effects. Common scenarios include:

- **Irreversible operations**: Data deletion, financial transactions, resource provisioning
- **External system access**: Database modifications, API calls, file system operations
- **Security-sensitive actions**: Credential access, permission changes, privileged operations

When a tool requires approval, the agent pauses execution and transfers control to the application. The agent preserves its state (conversation history, pending actions, context) and waits for the application's decision before proceeding. This mechanism ensures critical operations require explicit human authorization while maintaining the agent's execution context for immediate resumption.

As covered in Chapter 1, **Agent Middleware** provides a general-purpose mechanism for intercepting agent interactions at three levels: agent (intercepts entire agent execution), function (intercepts individual tool invocations), and chat (intercepts LLM calls). You can think of the process of user approval during tool invocation as *tool middleware*. 

TODO1 - requests and responses:
https://learn.microsoft.com/en-us/agent-framework/tutorials/workflows/requests-and-responses?pivots=programming-language-python


TODO2 - https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/workflows/agents/workflow_as_agent_human_in_the_loop.py

TODO3 - https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/workflows/human-in-the-loop/guessing_game_with_human_input.py
